In [ ]:
import os
import pickle
import pandas as pd
from pathlib import Path
from pku_autonomous_driving import io, util, dataset, resnet, centernet, training, graphics, transform, const, geometry

import importlib
importlib.reload(geometry)
importlib.reload(io)
importlib.reload(util)
importlib.reload(dataset)
importlib.reload(resnet)
importlib.reload(centernet)
importlib.reload(training)
importlib.reload(graphics)
importlib.reload(transform)
importlib.reload(const)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision
from pku_autonomous_driving.transform import CropBottomHalf, CropFar, PadByMean, Resize, Normalize, DropPointsAtOutOfScreen, CreateMaskAndRegr, ToCHWOrder
from pku_autonomous_driving.const import IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE

near_transform = torchvision.transforms.Compose([
    CropBottomHalf(),
    #PadByMean(),
    Resize(IMG_WIDTH, IMG_HEIGHT),
    Normalize(),
    DropPointsAtOutOfScreen(IMG_WIDTH, IMG_HEIGHT),
    CreateMaskAndRegr(IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE),
    ToCHWOrder()
])

far_transform = torchvision.transforms.Compose([
    CropFar(IMG_WIDTH, IMG_HEIGHT),
    Normalize(),
    DropPointsAtOutOfScreen(IMG_WIDTH, IMG_HEIGHT),
    CreateMaskAndRegr(IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE),
    ToCHWOrder()
])

transforms = {
    'NEAR': near_transform,
    'FAR': far_transform
}

train_transform = transforms[os.environ.get("TRANSFORM_TYPE", "NEAR")]

In [ ]:
from pku_autonomous_driving.dataset import CarDataset, create_data_loader
from pku_autonomous_driving.const import BATCH_SIZE

train, dev = io.load_train_data()
train = train[:4]
dev = dev[:4]

train_dataset = CarDataset(train, transform=train_transform)
dev_dataset = CarDataset(dev, transform=train_transform)

train_loader = create_data_loader(train_dataset, batch_size=BATCH_SIZE)
dev_loader = create_data_loader(dev_dataset, batch_size=1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = train_loader.dataset[0]
img, mask, regr = data["img"], data["mask"], data["regr"]
plt.figure(figsize=(16,16))
plt.imshow(np.rollaxis(img, 0, 3))
plt.show()

plt.figure(figsize=(16,16))
plt.imshow(mask)
plt.show()

plt.figure(figsize=(16,16))
plt.imshow(regr[-2])
plt.show()

In [ ]:
from torch import optim
from apex import amp

base_model = resnet.resnext50_32x4d(pretrained=False)
model = centernet.CentResnet(base_model, 8)

optimizer = optim.AdamW(model.parameters(), lr=0.001)
#optimizer =  RAdam(model.parameters(), lr = 0.001)

#model, optimizer = amp.initialize(model, optimizer)
#model = util.BN_convert_float(model.half())
#model = util.network_to_half(model)
setup_kwargs = {
    "model": model,
    "device": device,
    "path" : Path("./res_mask_1/resnext50.pth")
}

util.setup_model(**setup_kwargs)
model = model.float()

In [ ]:
n_epochs = int(os.environ.get("N_EPOCHS", 6))
n_epochs = 1

try:
    history = pickle.load(Path(os.environ["INITIAL_HISTORY"]).open('rb'))
    beg_epoch = math.ceil(history.index[-1])
except:
    history = pd.DataFrame()
    beg_epoch = 0
end_epoch = beg_epoch + n_epochs

In [ ]:
%%time
from torch.optim import lr_scheduler
import pandas as pd
import pickle

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=max(n_epochs, 10) * len(train_loader) // 3, gamma=0.1)

best_dev_loss = np.inf
for epoch in range(beg_epoch, end_epoch):
    training.clean_up()
    training.train(model, optimizer, exp_lr_scheduler, train_loader, epoch, device, history)
    training.evaluate(model, dev_loader, epoch, device, history)
    training.save_checkpoint(model, optimizer, history)

    cur_dev_loss = history['dev_loss'].dropna().iloc[-1]
    if cur_dev_loss < best_dev_loss:
        torch.save({"model": model.state_dict()}, './resnext50.pth')
        best_dev_loss = cur_dev_loss

In [ ]:
history['train_loss'].iloc[:].plot();

In [ ]:
series1 = history.dropna()['mask_loss']
plt.plot(series1.index, series1 ,label = 'mask loss');
series2 = history.dropna()['regr_loss']
plt.plot(series2.index, 30*series2,label = 'regr loss');
series3 = history.dropna()['dev_loss']
plt.plot(series3.index, series3,label = 'dev loss');
plt.show()

In [ ]:
series = history.dropna()['dev_loss']
plt.scatter(series.index, series);

In [ ]:
data = train_loader.dataset[0]
img, mask, regr = data["img"], data["mask"], data["regr"]

model.eval()

plt.figure(figsize=(16,16))
plt.title('Input image')
plt.imshow(np.rollaxis(img, 0, 3))
plt.show()

plt.figure(figsize=(16,16))
plt.title('Ground truth mask')
plt.imshow(mask)
plt.show()

output = model(torch.tensor(img[None]).to(device))
logits = output[0,0].data.cpu().numpy()

plt.figure(figsize=(16,16))
plt.title('Model predictions')
plt.imshow(logits)
plt.show()

plt.figure(figsize=(16,16))
plt.title('Model predictions thresholded')
plt.imshow(logits > 0)
plt.show()

In [ ]:
import gc
gc.collect()

training.clean_up()
for idx in range(4):
    data = dev_loader.dataset[idx]
    img, mask, regr = data["img"], data["mask"], data["regr"]
    output = model(torch.tensor(img[None]).to(device)).data.cpu().numpy()

    coords_pred = util.extract_coords(data, output[0])
    coords_true = util.extract_coords(data)

    img = io.load_image(dev_loader.dataset.dataset[idx].image_id)
    fig, axes = plt.subplots(1, 2, figsize=(30,30))
    axes[0].set_title('Ground truth')
    axes[0].imshow(graphics.draw_coords(img, coords_true))
    axes[1].set_title('Prediction')
    axes[1].imshow(graphics.draw_coords(img, coords_pred))
    plt.show()

In [ ]:
d = train_loader.dataset[0]["data"][6]
id = train_loader.dataset.dataset[0].image_id
d

In [ ]:
p = geometry.proj_world_to_screen(np.array([[d['x'], d['y'], d['z']]])).astype(np.int)

In [ ]:
img = io.load_image(id)
img[p[0,1]-50:p[0,1]+50, p[0,0]-50:p[0,0]+50,:] = (255, 0,0)

In [ ]:
plt.imshow(img)

In [ ]:
cm = io.load_camera_matrix()

In [ ]:
cm

In [ ]:
fov = 2 * math.atan(img.shape[1] / (2 * cm[0,0]))
fov

In [ ]:
fov / math.pi * 180

In [ ]:
d

In [ ]:
dp = np.array([[d['x'], d["y"], d['z']]])

In [ ]:
proj_dp = geometry.proj_world_to_screen(dp).astype(np.int)

In [ ]:
proj_dp

In [ ]:
math.pi / 2 - math.atan((proj_dp[0, 0] - img.shape[1] // 2) / cm[0, 0])

In [ ]:
diff = proj_dp[0,0] - img.shape[1] // 2

In [ ]:
ray_theta = math.pi / 2 + math.atan(diff / cm[0,0])
ray_theta

In [ ]:
ray_theta / math.pi * 180

In [ ]:
ray_pitch = calc_ray_pitch(proj_dp[0, 0], img.shape[1])

In [ ]:
ray_pitch / math.pi * 180

In [ ]:
def clamp_radian(theta, minv=0, maxv = (2 * math.pi)):
    if theta < minv:
        return clamp_radian(theta + 2 * math.pi, minv, maxv)
    elif maxv <= theta:
        return clamp_radian(theta - 2 * math.pi, minv, maxv)
    return theta

def calc_global_pitch(org_pitch):
    return clamp_radian(org_pitch + math.pi / 2)

def calc_org_pitch(global_pitch):
    return clamp_radian(global_pitch - math.pi / 2, -math.pi, math.pi)

def calc_ray_pitch(x, img_width, camera_matrix=io.load_camera_matrix()):
    diff = x - img_width // 2
    ray_pitch = math.pi / 2 + math.atan(diff / cm[0,0])
    return ray_pitch

In [ ]:
global_pitch = calc_global_pitch(d["pitch"])

In [ ]:
pitch_offset = global_pitch - ray_pitch

In [ ]:
calc_pitch_to_ray(d["pitch"], proj_dp[0, 0], img.shape[1])

In [ ]:
gp = calc_global_pitch(d["pitch"])
rp = calc_ray_pitch(proj_dp[0, 0], img.shape[1])
pitch_to_ray = gp - rp

In [ ]:
gp2 = pitch_to_ray + rp

In [ ]:
calc_org_pitch(gp2)

In [ ]:
d["pitch"]

In [ ]:
gp2

In [ ]:
gp

In [ ]:
rp

In [ ]:
math.pi / 2 - math.atan(d["x"] / d["z"])

In [ ]:
calc_ray_pitch(img.shape[1], img.shape[1]) / math.pi * 180

In [ ]:
calc_ray_pitch(0, img.shape[1]) / math.pi * 180

In [ ]:
126.28613099510488 + 53.7138690048951

In [ ]:
math.pi / 2 - math.atan((0 - img.shape[1] // 2) / cm[0, 0])

In [ ]:
math.atan(- img.shape[1] / (2 * cm[0, 0])) / math.pi * 180 * 2

In [ ]:
2 * math.atan(img.shape[1] / (2 * cm[0,0])) / math.pi * 180